In [3]:
import json
import logging
from functools import wraps
from typing import List, Dict
import tiktoken
from typing import List, Callable, Dict, Any
from prettytable import PrettyTable, ALL
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def log_operation(func):
    """
    Decorador para registrar las operaciones realizadas en la clase CompetitorSites.
    """
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        logger.info(f"Ejecutando {func.__name__} con args: {args}, kwargs: {kwargs}")
        result = func(self, *args, **kwargs)
        logger.info(f"{func.__name__} completado")
        return result
    return wrapper

In [5]:
class CompetitorSites:
    """
    Clase para manejar la información de sitios web competidores.

    Esta clase proporciona métodos para cargar, guardar y añadir sitios competidores
    desde y hacia un archivo JSON.

    Attributes:
        filename (str): Nombre del archivo JSON para almacenar los datos.
        sites (List[Dict[str, str]]): Lista de sitios competidores.
    """

    def __init__(self, filename: str):
        """
        Inicializa la instancia de CompetitorSites.

        Args:
            filename (str): Nombre del archivo JSON para almacenar los datos.
        """
        self.filename = filename
        self.sites = self.load_sites()
        logger.info(f"CompetitorSites inicializado con el archivo {filename}")

    @log_operation
    def load_sites(self) -> List[Dict[str, str]]:
        """
        Carga los sitios competidores desde el archivo JSON.

        Returns:
            List[Dict[str, str]]: Lista de sitios competidores.
        """
        try:
            with open(self.filename, 'r') as file:
                sites = json.load(file)
            logger.info(f"Sitios cargados exitosamente desde {self.filename}")
            return sites
        except FileNotFoundError:
            logger.warning(f"Archivo {self.filename} no encontrado. Iniciando con lista vacía.")
            return []
        except json.JSONDecodeError:
            logger.error(f"Error al decodificar JSON desde {self.filename}. Iniciando con lista vacía.")
            return []

    @log_operation
    def save_sites(self):
        """
        Guarda los sitios competidores en el archivo JSON.
        """
        try:
            with open(self.filename, 'w') as file:
                json.dump(self.sites, file, indent=4)
            logger.info(f"Sitios guardados exitosamente en {self.filename}")
        except IOError:
            logger.error(f"Error al guardar sitios en {self.filename}")

    @log_operation
    def add_site(self, name: str, url: str):
        """
        Añade un nuevo sitio competidor a la lista y guarda los cambios.

        Args:
            name (str): Nombre del sitio competidor.
            url (str): URL del sitio competidor.
        """
        new_site = {"name": name, "url": url}
        self.sites.append(new_site)
        self.save_sites()
        logger.info(f"Sitio añadido: {new_site}")

    def get_sites(self) -> List[Dict[str, str]]:
        """
        Obtiene la lista de todos los sitios competidores.

        Returns:
            List[Dict[str, str]]: Lista de sitios competidores.
        """
        return self.sites

In [6]:
class TokenCostCalculator:
    """
    Clase para calcular el costo de tokenización de texto.

    Esta clase proporciona métodos para contar tokens y calcular el costo
    asociado basado en un modelo de precios por millón de tokens.

    Attributes:
        cost_per_million_tokens (float): Costo por millón de tokens.
        tokenizer: Instancia del tokenizador de tiktoken.
    """

    def __init__(self, cost_per_million_tokens: float = 5):
        """
        Inicializa la instancia de TokenCostCalculator.

        Args:
            cost_per_million_tokens (float): Costo por millón de tokens. Por defecto es 5.
        """
        self.cost_per_million_tokens = cost_per_million_tokens
        try:
            self.tokenizer = tiktoken.get_encoding("cl100k_base")
            logger.info("Tokenizador inicializado correctamente")
        except Exception as e:
            logger.error(f"Error al inicializar el tokenizador: {e}")
            raise

    @log_operation
    def count_tokens(self, input_string: str) -> int:
        """
        Cuenta el número de tokens en una cadena de entrada.

        Args:
            input_string (str): Cadena de texto a tokenizar.

        Returns:
            int: Número de tokens en la cadena de entrada.
        """
        try:
            tokens = self.tokenizer.encode(input_string)
            return len(tokens)
        except Exception as e:
            logger.error(f"Error al contar tokens: {e}")
            raise

    @log_operation
    def calculate_cost(self, input_string: str) -> float:
        """
        Calcula el costo de tokenización para una cadena de entrada.

        Args:
            input_string (str): Cadena de texto para calcular el costo.

        Returns:
            float: Costo calculado para la tokenización de la cadena de entrada.
        """
        try:
            num_tokens = self.count_tokens(input_string)
            total_cost = (num_tokens / 1_000_000) * self.cost_per_million_tokens
            return total_cost
        except Exception as e:
            logger.error(f"Error al calcular el costo: {e}")
            raise

In [7]:
# Ejemplo de uso:

# Crear y usar el calculador de costos
calculator = TokenCostCalculator()
input_string = "What's the difference between beer nuts and deer nuts? Beer nuts are about 5 dollars. Deer nuts are just under a buck."
cost = calculator.calculate_cost(input_string)
print(f"El costo total por usar gpt-4o es: $US {cost:.6f}")

2024-07-27 11:35:09,432 - __main__ - INFO - Tokenizador inicializado correctamente
2024-07-27 11:35:09,433 - __main__ - INFO - Ejecutando calculate_cost con args: ("What's the difference between beer nuts and deer nuts? Beer nuts are about 5 dollars. Deer nuts are just under a buck.",), kwargs: {}
2024-07-27 11:35:09,433 - __main__ - INFO - Ejecutando count_tokens con args: ("What's the difference between beer nuts and deer nuts? Beer nuts are about 5 dollars. Deer nuts are just under a buck.",), kwargs: {}
2024-07-27 11:35:09,435 - __main__ - INFO - count_tokens completado
2024-07-27 11:35:09,435 - __main__ - INFO - calculate_cost completado


El costo total por usar gpt-4o es: $US 0.000135


In [8]:
# Crear y usar el manejador de sitios competidores
competitors = CompetitorSites("../data/competitor_sites.json")
    
# Agregar sitios competidores
competitors.add_site("Articulate 360 by Adobe", "https://www.articulate.com/360/pricing/freelancers")
competitors.add_site("7taps", "https://www.7taps.com/pricing")
competitors.add_site("Mindsmith AI", "https://www.mindsmith.ai/pricing")
competitors.add_site("Cards-microlearning", "https://www.cards-microlearning.com/en/tarifs")

print("Sitios competidores guardados en 'competitor_sites.json'")

2024-07-27 11:36:10,510 - __main__ - INFO - Ejecutando load_sites con args: (), kwargs: {}
2024-07-27 11:36:10,513 - __main__ - INFO - Sitios cargados exitosamente desde ../data/competitor_sites.json
2024-07-27 11:36:10,514 - __main__ - INFO - load_sites completado
2024-07-27 11:36:10,515 - __main__ - INFO - CompetitorSites inicializado con el archivo ../data/competitor_sites.json
2024-07-27 11:36:10,516 - __main__ - INFO - Ejecutando add_site con args: ('Articulate 360 by Adobe', 'https://www.articulate.com/360/pricing/freelancers'), kwargs: {}
2024-07-27 11:36:10,517 - __main__ - INFO - Ejecutando save_sites con args: (), kwargs: {}
2024-07-27 11:36:10,519 - __main__ - INFO - Sitios guardados exitosamente en ../data/competitor_sites.json
2024-07-27 11:36:10,520 - __main__ - INFO - save_sites completado
2024-07-27 11:36:10,520 - __main__ - INFO - Sitio añadido: {'name': 'Articulate 360 by Adobe', 'url': 'https://www.articulate.com/360/pricing/freelancers'}
2024-07-27 11:36:10,520 - __

Sitios competidores guardados en 'competitor_sites.json'


In [9]:
class ContentScraper:
    """
    Clase para scrapear contenido de sitios web y calcular costos asociados.

    Esta clase proporciona métodos para scrapear múltiples sitios web,
    calcular costos de tokenización y mostrar resultados en tablas formateadas.

    Attributes:
        scrape_url_functions (List[Dict[str, Callable]]): Funciones de scraping.
        sites_list (List[Dict[str, str]]): Lista de sitios a scrapear.
        token_cost_calculator: Calculadora de costos de tokenización.
        characters_to_display (int): Número de caracteres a mostrar en la tabla.
        table_max_width (int): Ancho máximo de las tablas.
    """

    def __init__(self, scrape_url_functions: List[Dict[str, Callable[[str], str]]], 
                 sites_list: List[Dict[str, str]], 
                 token_cost_calculator: Any,
                 characters_to_display: int = 500, 
                 table_max_width: int = 50):
        """
        Inicializa la instancia de ContentScraper.

        Args:
            scrape_url_functions: Lista de funciones de scraping.
            sites_list: Lista de sitios a scrapear.
            token_cost_calculator: Instancia de TokenCostCalculator.
            characters_to_display: Número de caracteres a mostrar en la tabla.
            table_max_width: Ancho máximo de las tablas.
        """
        self.scrape_url_functions = scrape_url_functions
        self.sites_list = sites_list
        self.token_cost_calculator = token_cost_calculator
        self.characters_to_display = characters_to_display
        self.table_max_width = table_max_width
        self.content_table = PrettyTable()
        self.cost_table = PrettyTable()
        self.scraped_data = []
        logger.info("ContentScraper inicializado")

    @log_operation
    def setup_tables(self):
        """Configura las tablas para mostrar contenido y costos."""
        content_table_headers = ["Site Name"] + [f"{func['name']} content" for func in self.scrape_url_functions]
        cost_table_headers = ["Site Name"] + [f"{func['name']} cost" for func in self.scrape_url_functions]
        self.content_table.field_names = content_table_headers
        self.cost_table.field_names = cost_table_headers
        self.content_table.max_width = self.table_max_width
        self.content_table.hrules = ALL
        self.cost_table.max_width = self.table_max_width
        self.cost_table.hrules = ALL

    @log_operation
    def scrape_site(self, site: Dict[str, str]) -> Dict[str, Any]:
        """
        Scrapea un sitio individual y calcula los costos asociados.

        Args:
            site: Diccionario con información del sitio a scrapear.

        Returns:
            Diccionario con los datos scrapeados y costos calculados.
        """
        content_row = [site['name']]
        cost_row = [site['name']]
        site_data = {"provider": site['name'], "sites": []}

        for scrape_function in self.scrape_url_functions:
            function_name = scrape_function['name']
            for _ in tqdm([site], desc=f"Processing site {site['name']} using {function_name}"):
                try:
                    content = scrape_function['function'](site['url'])
                    content_snippet = content[:self.characters_to_display]
                    content_row.append(content_snippet)
                    cost = self.token_cost_calculator.calculate_cost(content)
                    cost_row.append(f"${cost:.6f}")
                    site_data["sites"].append({"name": function_name, "content": content})
                    logger.info(f"Scrapeado exitosamente {site['name']} con {function_name}")
                except Exception as e:
                    error_message = f"Error: {str(e)}"
                    content_row.append(error_message)
                    cost_row.append("Error")
                    site_data["sites"].append({"name": function_name, "content": error_message})
                    logger.error(f"Error al scrapear {site['name']} con {function_name}: {e}")

        self.content_table.add_row(content_row)
        self.cost_table.add_row(cost_row)
        return site_data

    @log_operation
    def scrape_all_sites(self):
        """Scrapea todos los sitios en la lista."""
        self.setup_tables()
        for site in self.sites_list:
            site_data = self.scrape_site(site)
            self.scraped_data.append(site_data)

    @log_operation
    def display_results(self):
        """Muestra los resultados del scraping en tablas formateadas."""
        print("Content Table:")
        print(self.content_table)
        print("\nCost Table:\nThis is how much it would cost to use gpt-4o to parse this content for extraction.")
        print(self.cost_table)

    def get_scraped_data(self) -> List[Dict[str, Any]]:
        """
        Obtiene los datos scrapeados.

        Returns:
            Lista de diccionarios con los datos scrapeados.
        """
        return self.scraped_data

In [10]:
# Ejemplo de uso:

# Asumiendo que tienes definidas las funciones de scraping y la lista de sitios
scrape_functions = [
    {"name": "Función 1", "function": lambda url: f"Contenido de {url}"},
    {"name": "Función 2", "function": lambda url: f"Otro contenido de {url}"}
]
sites = [
    {"name": "Sitio 1", "url": "https://ejemplo1.com"},
    {"name": "Sitio 2", "url": "https://ejemplo2.com"}
]

In [11]:
# Asumiendo que tienes una instancia de TokenCostCalculator

calculator = TokenCostCalculator()

scraper = ContentScraper(scrape_functions, sites, calculator)
scraper.scrape_all_sites()
scraper.display_results()

2024-07-27 11:36:32,445 - __main__ - INFO - Tokenizador inicializado correctamente
2024-07-27 11:36:32,447 - __main__ - INFO - ContentScraper inicializado
2024-07-27 11:36:32,448 - __main__ - INFO - Ejecutando scrape_all_sites con args: (), kwargs: {}
2024-07-27 11:36:32,449 - __main__ - INFO - Ejecutando setup_tables con args: (), kwargs: {}
2024-07-27 11:36:32,449 - __main__ - INFO - setup_tables completado
2024-07-27 11:36:32,449 - __main__ - INFO - Ejecutando scrape_site con args: ({'name': 'Sitio 1', 'url': 'https://ejemplo1.com'},), kwargs: {}
Processing site Sitio 1 using Función 1:   0%|          | 0/1 [00:00<?, ?it/s]2024-07-27 11:36:32,466 - __main__ - INFO - Ejecutando calculate_cost con args: ('Contenido de https://ejemplo1.com',), kwargs: {}
2024-07-27 11:36:32,466 - __main__ - INFO - Ejecutando count_tokens con args: ('Contenido de https://ejemplo1.com',), kwargs: {}
2024-07-27 11:36:32,468 - __main__ - INFO - count_tokens completado
2024-07-27 11:36:32,468 - __main__ - I

Content Table:
+-----------+-----------------------------------+----------------------------------------+
| Site Name |         Función 1 content         |           Función 2 content            |
+-----------+-----------------------------------+----------------------------------------+
|  Sitio 1  | Contenido de https://ejemplo1.com | Otro contenido de https://ejemplo1.com |
+-----------+-----------------------------------+----------------------------------------+
|  Sitio 2  | Contenido de https://ejemplo2.com | Otro contenido de https://ejemplo2.com |
+-----------+-----------------------------------+----------------------------------------+

Cost Table:
This is how much it would cost to use gpt-4o to parse this content for extraction.
+-----------+----------------+----------------+
| Site Name | Función 1 cost | Función 2 cost |
+-----------+----------------+----------------+
|  Sitio 1  |   $0.000045    |   $0.000050    |
+-----------+----------------+----------------+
|  Sitio 2  

In [12]:
# Si necesitas los datos crudos
scraped_data = scraper.get_scraped_data()
print(scraped_data)

[{'provider': 'Sitio 1', 'sites': [{'name': 'Función 1', 'content': 'Contenido de https://ejemplo1.com'}, {'name': 'Función 2', 'content': 'Otro contenido de https://ejemplo1.com'}]}, {'provider': 'Sitio 2', 'sites': [{'name': 'Función 1', 'content': 'Contenido de https://ejemplo2.com'}, {'name': 'Función 2', 'content': 'Otro contenido de https://ejemplo2.com'}]}]


In [13]:
class Scraper:
    """
    Clase para realizar scraping de sitios web utilizando diferentes métodos.

    Esta clase proporciona métodos para scrapear sitios web utilizando BeautifulSoup
    y Jina AI, y permite añadir funciones de scraping personalizadas.

    Attributes:
        scrape_functions (List[Dict[str, Callable]]): Lista de funciones de scraping disponibles.
    """

    def __init__(self):
        """
        Inicializa la instancia de Scraper con funciones de scraping predefinidas.
        """
        self.scrape_functions = [
            {"name": "BeautifulSoup", "function": self.beautiful_soup_scrape_url},
            {"name": "JinaAI", "function": self.scrape_jina_ai}
        ]
        logger.info("Scraper inicializado con funciones predefinidas")

    @log_operation
    def beautiful_soup_scrape_url(self, url: str) -> str:
        """
        Scrapea una URL utilizando BeautifulSoup.

        Args:
            url (str): La URL a scrapear.

        Returns:
            str: El contenido HTML de la página scrapeada.

        Raises:
            requests.RequestException: Si ocurre un error al hacer la solicitud HTTP.
        """
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            return str(soup)
        except requests.RequestException as e:
            logger.error(f"Error al scrapear {url} con BeautifulSoup: {e}")
            raise

    @log_operation
    def scrape_jina_ai(self, url: str) -> str:
        """
        Scrapea una URL utilizando Jina AI.

        Args:
            url (str): La URL a scrapear.

        Returns:
            str: El contenido de texto de la página scrapeada.

        Raises:
            requests.RequestException: Si ocurre un error al hacer la solicitud HTTP.
        """
        try:
            response = requests.get("https://r.jina.ai/" + url)
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            logger.error(f"Error al scrapear {url} con Jina AI: {e}")
            raise

    @log_operation
    def get_scrape_functions(self) -> List[Dict[str, Callable[[str], str]]]:
        """
        Obtiene la lista de funciones de scraping disponibles.

        Returns:
            List[Dict[str, Callable[[str], str]]]: Lista de funciones de scraping.
        """
        return self.scrape_functions

    @log_operation
    def add_scrape_function(self, name: str, function: Callable[[str], str]):
        """
        Añade una nueva función de scraping a la lista de funciones disponibles.

        Args:
            name (str): Nombre de la nueva función de scraping.
            function (Callable[[str], str]): La función de scraping a añadir.
        """
        self.scrape_functions.append({"name": name, "function": function})
        logger.info(f"Nueva función de scraping añadida: {name}")

In [14]:
# Ejemplo de uso
scraper = Scraper()

2024-07-27 11:36:51,154 - __main__ - INFO - Scraper inicializado con funciones predefinidas


In [15]:
# Ejemplo de uso de BeautifulSoup
try:
    content = scraper.beautiful_soup_scrape_url("https://www.articulate.com/360/pricing/freelancers")
    print("Contenido scrapeado con BeautifulSoup:")
    print(content[:500])  # Primeros 500 caracteres
except requests.RequestException as e:
    print(f"Error al scrapear con BeautifulSoup: {e}")

2024-07-27 11:36:53,493 - __main__ - INFO - Ejecutando beautiful_soup_scrape_url con args: ('https://www.articulate.com/360/pricing/freelancers',), kwargs: {}
2024-07-27 11:36:53,721 - __main__ - ERROR - Error al scrapear https://www.articulate.com/360/pricing/freelancers con BeautifulSoup: 403 Client Error: Forbidden for url: https://www.articulate.com/360/pricing/freelancers


Error al scrapear con BeautifulSoup: 403 Client Error: Forbidden for url: https://www.articulate.com/360/pricing/freelancers


In [16]:
# Ejemplo de uso de Jina AI
try:
    content = scraper.scrape_jina_ai("https://www.articulate.com/360/pricing/freelancers")
    print("\nContenido scrapeado con Jina AI:")
    print(content[:500])  # Primeros 500 caracteres
except requests.RequestException as e:
    print(f"Error al scrapear con Jina AI: {e}")

2024-07-27 11:37:02,315 - __main__ - INFO - Ejecutando scrape_jina_ai con args: ('https://www.articulate.com/360/pricing/freelancers',), kwargs: {}
2024-07-27 11:37:05,525 - __main__ - INFO - scrape_jina_ai completado



Contenido scrapeado con Jina AI:
Title: Freelancer Pricing for Articulate 360 - Everything You Need to Create E‑Learning

URL Source: https://www.articulate.com/360/pricing/freelancers

Markdown Content:
Our personal plan includes everything you need to create e-learning on your own.

Articulate 360 for Freelancers
------------------------------

![Image 1](https://www.articulate.com/wp-content/uploads/2023/10/pricing-illo.png)

[Personal Plan](https://www.articulate.com/360/)
------------------------------------------------

*


In [17]:
# Añadir una nueva función de scraping
scraper.add_scrape_function("Custom", lambda url: f"Contenido personalizado de {url}")

# Listar todas las funciones de scraping
print("\nFunciones de scraping disponibles:")
for func in scraper.get_scrape_functions():
    print(f"- {func['name']}")

2024-07-27 11:37:29,726 - __main__ - INFO - Ejecutando add_scrape_function con args: ('Custom', <function <lambda> at 0x113532200>), kwargs: {}
2024-07-27 11:37:29,727 - __main__ - INFO - Nueva función de scraping añadida: Custom
2024-07-27 11:37:29,728 - __main__ - INFO - add_scrape_function completado
2024-07-27 11:37:29,728 - __main__ - INFO - Ejecutando get_scrape_functions con args: (), kwargs: {}
2024-07-27 11:37:29,729 - __main__ - INFO - get_scrape_functions completado



Funciones de scraping disponibles:
- BeautifulSoup
- JinaAI
- Custom


In [18]:
# Lista de sitios competidores
competitor_sites = [
    {"name": "Articulate 360 by Adobe", "url": "https://www.articulate.com/360/pricing/freelancers"},
    {"name": "7taps", "url": "https://www.7taps.com/pricing"},
    {"name": "Mindsmith AI", "url": "https://www.mindsmith.ai/pricing"},
    {"name": "Cards-microlearning", "url": "https://www.cards-microlearning.com/en/tarifs"},
]

In [19]:
def run_all_scrapers(sites: List[Dict[str, str]], characters_to_display: int = 700, table_max_width: int = 20):
    """
    Ejecuta el proceso de scraping para todos los sitios especificados.

    Args:
        sites (List[Dict[str, str]]): Lista de sitios a scrapear.
        characters_to_display (int): Número de caracteres a mostrar en la tabla de resultados.
        table_max_width (int): Ancho máximo de la tabla de resultados.

    Returns:
        List[Dict]: Datos crudos obtenidos del scraping.
    """
    logger.info("Iniciando proceso de scraping para todos los sitios")

    try:
        # Inicializar el Scraper
        scraper = Scraper()
        logger.info("Scraper inicializado")

        # Obtener las funciones de scraping
        scrape_functions = scraper.get_scrape_functions()
        logger.info(f"Funciones de scraping obtenidas: {[func['name'] for func in scrape_functions]}")

        # Inicializar el TokenCostCalculator
        token_calculator = TokenCostCalculator()
        logger.info("TokenCostCalculator inicializado")

        # Inicializar el ContentScraper
        content_scraper = ContentScraper(scrape_functions, sites, token_calculator, characters_to_display, table_max_width)
        logger.info("ContentScraper inicializado")

        # Ejecutar el scraping para todos los sitios
        content_scraper.scrape_all_sites()
        logger.info("Scraping completado para todos los sitios")

        # Mostrar los resultados
        content_scraper.display_results()
        logger.info("Resultados mostrados")

        # Obtener los datos crudos
        scraped_data = content_scraper.get_scraped_data()
        logger.info("Datos crudos obtenidos")

        return scraped_data

    except Exception as e:
        logger.error(f"Error durante el proceso de scraping: {e}")
        raise

In [20]:
# Ejecutar el scraping y mostrar los resultados

try:
    print("Ejecutando todos los scrapers y mostrando resultados...")
    all_content = run_all_scrapers(competitor_sites)
        
    print("\nDatos crudos obtenidos:")
    for site_data in all_content:
        print(f"\nProveedor: {site_data['provider']}")
        for site in site_data['sites']:
            print(f"  Scraper: {site['name']}")
            print(f"  Contenido: {site['content'][:100]}...")  # Mostrar solo los primeros 100 caracteres

    logger.info("Proceso de scraping completado exitosamente")
except Exception as e:
    logger.error(f"Error en el proceso principal: {e}")
    print(f"Ocurrió un error: {e}")

2024-07-27 11:37:37,770 - __main__ - INFO - Iniciando proceso de scraping para todos los sitios
2024-07-27 11:37:37,771 - __main__ - INFO - Scraper inicializado con funciones predefinidas
2024-07-27 11:37:37,771 - __main__ - INFO - Scraper inicializado
2024-07-27 11:37:37,772 - __main__ - INFO - Ejecutando get_scrape_functions con args: (), kwargs: {}
2024-07-27 11:37:37,772 - __main__ - INFO - get_scrape_functions completado
2024-07-27 11:37:37,773 - __main__ - INFO - Funciones de scraping obtenidas: ['BeautifulSoup', 'JinaAI']
2024-07-27 11:37:37,773 - __main__ - INFO - Tokenizador inicializado correctamente
2024-07-27 11:37:37,773 - __main__ - INFO - TokenCostCalculator inicializado
2024-07-27 11:37:37,774 - __main__ - INFO - ContentScraper inicializado
2024-07-27 11:37:37,774 - __main__ - INFO - ContentScraper inicializado
2024-07-27 11:37:37,774 - __main__ - INFO - Ejecutando scrape_all_sites con args: (), kwargs: {}
2024-07-27 11:37:37,775 - __main__ - INFO - Ejecutando setup_tab

Ejecutando todos los scrapers y mostrando resultados...


Processing site Articulate 360 by Adobe using BeautifulSoup:   0%|          | 0/1 [00:00<?, ?it/s]2024-07-27 11:37:37,778 - __main__ - INFO - Ejecutando beautiful_soup_scrape_url con args: ('https://www.articulate.com/360/pricing/freelancers',), kwargs: {}
2024-07-27 11:37:37,939 - __main__ - ERROR - Error al scrapear https://www.articulate.com/360/pricing/freelancers con BeautifulSoup: 403 Client Error: Forbidden for url: https://www.articulate.com/360/pricing/freelancers
2024-07-27 11:37:37,940 - __main__ - ERROR - Error al scrapear Articulate 360 by Adobe con BeautifulSoup: 403 Client Error: Forbidden for url: https://www.articulate.com/360/pricing/freelancers
Processing site Articulate 360 by Adobe using BeautifulSoup: 100%|██████████| 1/1 [00:00<00:00,  6.08it/s]
Processing site Articulate 360 by Adobe using JinaAI:   0%|          | 0/1 [00:00<?, ?it/s]2024-07-27 11:37:37,945 - __main__ - INFO - Ejecutando scrape_jina_ai con args: ('https://www.articulate.com/360/pricing/freelance

Content Table:
+----------------------+-----------------------+----------------------+
|      Site Name       | BeautifulSoup content |    JinaAI content    |
+----------------------+-----------------------+----------------------+
|  Articulate 360 by   |   Error: 403 Client   |  Title: Freelancer   |
|        Adobe         |  Error: Forbidden for |     Pricing for      |
|                      | url: https://www.arti |   Articulate 360 -   |
|                      | culate.com/360/pricin | Everything You Need  |
|                      |     g/freelancers     | to Create E‑Learning |
|                      |                       |                      |
|                      |                       | URL Source: https:// |
|                      |                       | www.articulate.com/3 |
|                      |                       | 60/pricing/freelance |
|                      |                       |          rs          |
|                      |                       | 

In [21]:
class OpenAIHandler:
    """
    Manejador para interactuar con la API de OpenAI.

    Esta clase proporciona métodos para inicializar la conexión con OpenAI
    y realizar llamadas a la API para obtener completaciones.

    Attributes:
        api_key (str): Clave API de OpenAI.
        client (OpenAI): Cliente de OpenAI inicializado.
    """

    @log_operation
    def __init__(self):
        """
        Inicializa la instancia de OpenAIHandler.

        Carga la clave API desde un archivo .env y configura el cliente de OpenAI.

        Raises:
            ValueError: Si no se encuentra la clave API de OpenAI en el archivo .env.
        """
        load_dotenv()
        self.api_key = os.getenv('OPENAI_API_KEY')
        if not self.api_key:
            logger.error("No se encontró la clave API de OpenAI en el archivo .env")
            raise ValueError("No se encontró la clave API de OpenAI. Asegúrate de tener un archivo .env con OPENAI_API_KEY definido.")
        self.client = OpenAI(api_key=self.api_key)
        logger.info("OpenAIHandler inicializado correctamente")

    @log_operation
    def get_completion(self, messages: List[Dict[str, str]]) -> str:
        """
        Obtiene una completación de la API de OpenAI.

        Args:
            messages (List[Dict[str, str]]): Lista de mensajes para la conversación con la API.

        Returns:
            str: Contenido de la respuesta de la API en formato JSON.

        Raises:
            Exception: Si ocurre un error durante la llamada a la API.
        """
        try:
            logger.info(f"Realizando llamada a la API de OpenAI con {len(messages)} mensajes")
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                stream=False,
                response_format={"type": "json_object"}
            )
            logger.info("Llamada a la API completada exitosamente")
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error en la llamada a la API de OpenAI: {e}")
            return json.dumps({"error": str(e)})

In [22]:
class ContentProcessor:
    """
    Clase para procesar contenido y extraer información de precios.

    Esta clase proporciona métodos para dividir contenido en chunks,
    procesar el contenido con OpenAI y extraer información de precios.

    Attributes:
        openai_handler (OpenAIHandler): Instancia del manejador de OpenAI.
    """

    def __init__(self, openai_handler: OpenAIHandler):
        """
        Inicializa la instancia de ContentProcessor.

        Args:
            openai_handler (OpenAIHandler): Instancia del manejador de OpenAI.
        """
        self.openai_handler = openai_handler
        logger.info("ContentProcessor inicializado")

    @staticmethod
    @log_operation
    def chunk_content(content: str, max_tokens: int = 4000) -> List[str]:
        """
        Divide el contenido en chunks más pequeños.

        Args:
            content (str): Contenido a dividir.
            max_tokens (int): Número máximo de tokens por chunk.

        Returns:
            List[str]: Lista de chunks de contenido.
        """
        words = content.split()
        chunks = []
        current_chunk = []
        current_length = 0

        for word in words:
            if current_length + len(word.split()) > max_tokens:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
                current_length = 0
            current_chunk.append(word)
            current_length += len(word.split())

        if current_chunk:
            chunks.append(' '.join(current_chunk))

        logger.info(f"Contenido dividido en {len(chunks)} chunks")
        return chunks

    @staticmethod
    def get_valid_price(tier: Dict[str, Any]) -> float | None:
        """
        Obtiene un precio válido de un tier.

        Args:
            tier (Dict[str, Any]): Diccionario que representa un tier de precio.

        Returns:
            float | None: Precio válido o None si no se encuentra un precio válido.
        """
        return tier.get("price") if tier and isinstance(tier.get("price"), (int, float)) else None

    @log_operation
    def extract(self, user_input: str) -> str:
        """
        Extrae información de precios del contenido proporcionado.

        Args:
            user_input (str): Contenido del cual extraer información de precios.

        Returns:
            str: JSON string con la información de precios extraída.
        """
        entity_extraction_system_message = {
            "role": "system",
            "content": "Get me the three pricing tiers from this website's content, and return as a JSON with three keys: {cheapest: {name: str, price: float}, middle: {name: str, price: float}, most_expensive: {name: str, price: float}}. If you can't find a price, use null for the price value."
        }

        chunks = self.chunk_content(user_input)
        all_results = []

        for i, chunk in enumerate(chunks):
            logger.info(f"Procesando chunk {i+1}/{len(chunks)}")
            messages = [
                entity_extraction_system_message,
                {"role": "user", "content": chunk}
            ]
            try:
                result = json.loads(self.openai_handler.get_completion(messages))
                all_results.append(result)
            except json.JSONDecodeError:
                logger.error(f"Error al decodificar JSON para el chunk {i+1}")

        try:
            final_result = {
                "cheapest": min(
                    (r["cheapest"] for r in all_results if "cheapest" in r),
                    key=lambda x: self.get_valid_price(x) or float("inf")
                ),
                "most_expensive": max(
                    (r["most_expensive"] for r in all_results if "most_expensive" in r),
                    key=lambda x: self.get_valid_price(x) or float("-inf")
                )
            }

            all_prices = [
                tier for r in all_results for tier in [r.get("cheapest"), r.get("middle"), r.get("most_expensive")]
                if tier and self.get_valid_price(tier) is not None
            ]
            all_prices.sort(key=lambda x: self.get_valid_price(x))
            final_result["middle"] = all_prices[len(all_prices) // 2] if all_prices else None

            logger.info("Extracción de precios completada exitosamente")
            return json.dumps(final_result)
        except Exception as e:
            logger.error(f"Error al procesar los resultados finales: {e}")
            return json.dumps({"error": "No se pudo extraer la información de precios"})

In [23]:
# Ejemplo de uso

try:
    handler = OpenAIHandler()
    messages = [
        {"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "Dime un hecho interesante sobre Python en formato JSON."}
    ]
    result = handler.get_completion(messages)
    print("Respuesta de la API:")
    print(json.dumps(json.loads(result), indent=2))
except Exception as e:
    print(f"Error en la ejecución: {e}")

2024-07-27 11:38:25,523 - __main__ - INFO - Ejecutando __init__ con args: (), kwargs: {}
2024-07-27 11:38:25,534 - __main__ - INFO - OpenAIHandler inicializado correctamente
2024-07-27 11:38:25,534 - __main__ - INFO - __init__ completado
2024-07-27 11:38:25,535 - __main__ - INFO - Ejecutando get_completion con args: ([{'role': 'system', 'content': 'Eres un asistente útil.'}, {'role': 'user', 'content': 'Dime un hecho interesante sobre Python en formato JSON.'}],), kwargs: {}
2024-07-27 11:38:25,535 - __main__ - INFO - Realizando llamada a la API de OpenAI con 2 mensajes
2024-07-27 11:38:28,221 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-27 11:38:28,228 - __main__ - INFO - Llamada a la API completada exitosamente
2024-07-27 11:38:28,229 - __main__ - INFO - get_completion completado


Respuesta de la API:
{
  "hecho_interesante": {
    "titulo": "Python y su origen",
    "contenido": "Python fue creado a finales de la d\u00e9cada de 1980 por Guido van Rossum en los Pa\u00edses Bajos como una sucesora del lenguaje de programaci\u00f3n ABC.",
    "fecha": "1989",
    "creador": "Guido van Rossum",
    "prop\u00f3sito_inicial": "El lenguaje fue dise\u00f1ado para ser f\u00e1cil de aprender y utilizar, con un enfoque en la legibilidad del c\u00f3digo."
  }
}


In [24]:
# Ejemplo de uso

    
openai_handler = OpenAIHandler()
processor = ContentProcessor(openai_handler)
    
sample_content = """
Nuestros planes de precios:
Básico: $9.99/mes
Pro: $19.99/mes
Empresarial: $49.99/mes
"""
    
result = processor.extract(sample_content)
print("Resultado de la extracción:")
print(json.dumps(json.loads(result), indent=2))

2024-07-27 11:38:38,942 - __main__ - INFO - Ejecutando __init__ con args: (), kwargs: {}
2024-07-27 11:38:38,950 - __main__ - INFO - OpenAIHandler inicializado correctamente
2024-07-27 11:38:38,951 - __main__ - INFO - __init__ completado
2024-07-27 11:38:38,951 - __main__ - INFO - ContentProcessor inicializado
2024-07-27 11:38:38,952 - __main__ - INFO - Ejecutando extract con args: ('\nNuestros planes de precios:\nBásico: $9.99/mes\nPro: $19.99/mes\nEmpresarial: $49.99/mes\n',), kwargs: {}
2024-07-27 11:38:38,952 - __main__ - INFO - Ejecutando chunk_content con args: (), kwargs: {}
2024-07-27 11:38:38,952 - __main__ - INFO - Contenido dividido en 1 chunks
2024-07-27 11:38:38,952 - __main__ - INFO - chunk_content completado
2024-07-27 11:38:38,952 - __main__ - INFO - Procesando chunk 1/1
2024-07-27 11:38:38,953 - __main__ - INFO - Ejecutando get_completion con args: ([{'role': 'system', 'content': "Get me the three pricing tiers from this website's content, and return as a JSON with thr

Resultado de la extracción:
{
  "cheapest": {
    "name": "B\u00e1sico",
    "price": 9.99
  },
  "most_expensive": {
    "name": "Empresarial",
    "price": 49.99
  },
  "middle": {
    "name": "Pro",
    "price": 19.99
  }
}
